In [1]:
import json
import requests
import sys
import warnings
from datetime import datetime, time, timedelta
from html import escape
from selenium import webdriver
from selenium.webdriver.common.by import By
#from seleniumwire import webdriver

GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
sys.path.insert(0, '../../../ajna_docs/commons/')
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')
from virasana.integracao.due import raspa_due

warnings.simplefilter("ignore")

VIRASANA_URL = "https://ajna1.rfoc.srf/virasana/"

C:\Users\25052288840\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-03 10:46:05,514 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [2]:
ontem = datetime.today() - timedelta(days=1)
datainicial = datetime.strftime(datetime.combine(ontem, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(ontem, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2022-11-02  00:00:00 2022-11-02 23:59:59


In [3]:
def login():
    driver.get('https://portalunico.siscomex.gov.br/')
    id_governo = driver.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/div[1]/div[5]/div[1]')
    id_governo.click()
    span_rfb = driver.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/div[1]/div[5]/div[2]/ul/li[1]')
    span_rfb.click()
    id_login = driver.find_element(By.XPATH, '/html/body/div/div[1]/div[4]/div[2]')
    id_login.click()

In [4]:
def get_due_posacd(numero_container: str)-> str:
    driver.get('https://portalunico.siscomex.gov.br/cct/#/consulta-estoque-pos-acd')
    driver.refresh();
    option_container=driver.find_element(By.XPATH,
     '/html/body/plt-layout/div[1]/div[5]/div/div/pucx-tela-consulta/div/div[2]/div/form/div[1]/div/div/span/div/span[4]/label')
    option_container.click()
    txt_container = driver.find_element(By.XPATH,
     '/html/body/plt-layout/div[1]/div[5]/div/div/pucx-tela-consulta/div/div[2]/div/form/div[2]/div/div/div/div/span/pucx-container/input')
    txt_container.send_keys(numero_container)
    btn_consulta = driver.find_element(By.XPATH, 
     '/html/body/plt-layout/div[1]/div[5]/div/div/pucx-tela-consulta/div/div[2]/div/form/div[3]/button[1]')
    btn_consulta.click()
    div_feedback = driver.find_element(By.XPATH, '/html/body/plt-layout/div[1]/div[3]/div/div')
    # print(div_feedback.text)
    div_due = driver.find_element(By.XPATH, 
     '/html/body/plt-layout/div[1]/div[5]/div/div/div/div[2]/pucx-grid/div/div/div/div[1]/div[1]/div[2]/div/div/div[1]/div[3]/div')
    return div_due.text

In [5]:
conteineres_vazios_ids = raspa_due.raspa_containers_vazios_sem_due(datainicial, datafinal, virasana_url=VIRASANA_URL)
conteineres_lce_ids = raspa_due.raspa_containers_sem_due(datainicial, datafinal, 'lce', virasana_url=VIRASANA_URL)
conteineres_semnada_ids = raspa_due.raspa_containers_sem_due(datainicial, datafinal, '', virasana_url=VIRASANA_URL)


Conectando virasana
0 Contêineres recuperados.
Conectando virasana
1 Contêineres recuperados.
Conectando virasana
1035 Contêineres recuperados.


In [10]:
driver = webdriver.Chrome(GECKO_PATH)
driver.implicitly_wait(10)
login()
driver.implicitly_wait(3)

In [11]:
from selenium.common.exceptions import NoSuchElementException
def salva_due_conteineres(conteineres_ids):
    for container, _id in conteineres_ids.items():
        try:
            due = get_due_posacd(container)
            print(container, due)
            pacote_carregamento = {_id: [{'numero': due}] }
            r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento, verify=False)
            print(r.status_code)
            print(r.text)
        except NoSuchElementException:
            print(f'Contêiner {container} não localizado')

salva_due_conteineres(conteineres_lce_ids)

CMAU0805735 22BR0018752061
201
{"status":"DUEs inseridas/atualizadas"}



In [8]:
salva_due_conteineres(conteineres_vazios_ids)

In [9]:
salva_due_conteineres(conteineres_semnada_ids)

Contêiner ZIMU1393827 não localizado
Contêiner ERRO não localizado


AttributeError: 'NoneType' object has no attribute 'click'